In this part, we'll prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process.

## Read the data 

In [1]:
import pandas as pd
pd.options.display.max_columns = 99 

loans = pd.read_csv('./Data/filtered_loans_2007.csv')
null_counts = loans.isnull().sum()
null_counts

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

## Handling Missing Values

While most of the columns have no missing values, two columns have fifty or less rows with missing values, and two columns, emp_length and pub_rec_bankruptcies, contain a relatively high amount of missing values.

Domain knowledge tells us that employment length is frequently used in assessing how risky a potential borrower is, so we'll keep this column despite its relatively large amount of missing values.

Let's inspect the values of the column pub_rec_bankruptcies

In [2]:
loans['pub_rec_bankruptcies'].value_counts(normalize=True, dropna=False)

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64

We see that this column offers very little variability, nearly 94% of values are in the same category. It probably won't have much predictive value. let's drop it. In addition, we'll remove the remaining rows containing null values.

This means that we'll keep the following columns and just remove rows containing missing values for them:

* emp_length
* title
* revol_util
* last_credit_pull_d
and drop the `pub_rec_bankruptcies` column entirely.

In [3]:
loans = loans.drop('pub_rec_bankruptcies', axis=1)
loans = loans.dropna()
loans.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

## Text Columns 

While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth.

In [4]:
object_columns_df = loans.select_dtypes(include=['object'])
object_columns_df.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,Apr-1999,9.4%,Sep-2013
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,Nov-2001,98.5%,Jun-2016
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,Feb-1996,21%,Apr-2016
4,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,Nov-2004,28.3%,Jan-2016


### First 5 categorical columns

Let's explore the unique value counts of the columnns that seem like they contain categorical values.


In [5]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']

for col in cols:
    print(loans[col].value_counts())
    print('-'*12)

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
------------
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
------------
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
------------
 36 months    28234
 60 months     9441
Name: term, dtype: int64
------------
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
UT     249
KS     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     

The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

### The reason for the loan 

First, let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

In [6]:
loans['title'].value_counts()

Debt Consolidation                                                2068
Debt Consolidation Loan                                           1599
Personal Loan                                                      624
Consolidation                                                      488
debt consolidation                                                 466
Credit Card Consolidation                                          345
Home Improvement                                                   336
Debt consolidation                                                 314
Small Business Loan                                                298
Credit Card Loan                                                   294
Personal                                                           290
Consolidation Loan                                                 250
Home Improvement Loan                                              228
personal loan                                                      219
Loan  

In [7]:
loans['purpose'].value_counts()

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64

## Categorical columns

The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

In [8]:
# Remove the last_credit_pull_d, addr_state, title, and earliest_cr_line columns from loans
loans = loans.drop(['last_credit_pull_d','addr_state','title','earliest_cr_line'], axis=1)

# Convert the int_rate and revol_util columns to float columns
loans['int_rate'] = loans['int_rate'].str.rstrip('%').astype('float')
loans['revol_util'] = loans['revol_util'].str.rstrip('%').astype('float')

# clean the emp_length column
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.replace(mapping_dict)

## Dummy variables

Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy variables so we can use them in our model.

In [9]:
cols = ['home_ownership','verification_status','purpose','term']
# Create dummy variables 
dummy_df = pd.get_dummies(loans[cols])
# add the dummy columns to the dataframe
loans = pd.concat([loans, dummy_df], axis=1)
# remove the original columns from the dataframe
loans = loans.drop(cols, axis=1)

## Save the data 

In [10]:
loans.to_csv('./Data/cleaned_loans.csv', index=False)